In [2]:
import sys
sys.path.append('..')

In [1]:
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Matplotlib is building the font cache; this may take a moment.


In [3]:
from sklearn.metrics import mean_squared_error
from scipy import stats

In [4]:
from DomainPrediction.utils import helper
from DomainPrediction.eval import metrics
from DomainPrediction.al import top_model as topmodel
from DomainPrediction.al.embeddings import one_hot_encode
from DomainPrediction.protein.base import BaseProtein
from DomainPrediction.utils.constants import *

In [5]:
sys.path.append('../../esm')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein, LogitsConfig, GenerationConfig, ESMProteinTensor, ESMProteinError, SamplingTrackConfig, SamplingConfig
from esm.utils.structure.protein_chain import ProteinChain

from esm.utils.generation import (
    iterative_sampling_raw, 
    iterative_sampling_tokens, 
    _get_non_special_tokens, 
    _get_masked_positions,
    _stack_protein_tensors,
    _slice_tensor_dataclass,
    _trim_sequence_tensor_dataclass,
    _get_annealed_temperature,
    _sample_per_prompt,
    _get_iterative_sampling_mask_for_prompt_and_step
)

### Trial run on Online DPO

In [ ]:
# device  = 'gpu'
# if device == 'gpu':
#     model = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda")
# else:
#     model = ESM3.from_pretrained("esm3_sm_open_v1").to("cpu")

In [ ]:
# protein = ProteinChain.from_pdb('../../Data/gxps/gxps_ATC_AF.pdb')

# sequence_prompt = ''.join([protein[i].sequence if i in A_gxps_atc + C_gxps_atc else '_' for i in range(len(protein))])
# structure_prompt = torch.tensor(protein.atom37_positions)

# assert sequence_prompt.count('_') == 115

In [ ]:
# structure_prompt.shape

In [ ]:
# len(sequence_prompt)

In [ ]:
# sequence_prediction_config = GenerationConfig(
#         track="sequence", 
#         num_steps=sequence_prompt.count("_") // 2, 
#         temperature=0.5
#     )

# esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=None)
# # esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=structure_prompt)

# # generated_protein = model.generate(esm_protein, sequence_prediction_config)

In [ ]:
# # proteins = model.batch_generate([esm_protein], [sequence_prediction_config])

# '''
#     we are inside batch generate
# '''
# inputs = [esm_protein]
# configs = [sequence_prediction_config]

# assert len(inputs) == len(
#             configs
#         ), "Must have the same number of prompts and configs."

# if inputs == []:
#     raise Exception('Empty list')

# # Make sure prompts are of the same type.
# t = type(inputs[0])
# for i in range(1, len(inputs)):
#     assert isinstance(inputs[i], t), (
#         "Prompts must have the same type. Got "
#         f"{t.__name__ and type(inputs[i]).__name__} instead."
#     )

# if isinstance(inputs[0], ESMProtein):
#     print('We go with iterative_sampling_raw')
#     proteins =  iterative_sampling_raw(model, inputs, configs)  # type: ignore
# elif isinstance(inputs[0], ESMProteinTensor):
#     print('We go with iterative_sampling_tokens')
#     proteins =  iterative_sampling_tokens(
#         model,
#         inputs,  # type: ignore
#         configs,
#         model.tokenizers,  # type: ignore
#     )
# else:
#     raise ValueError("Input must be an ESMProtein or ESMProteinTensor")

# '''
#     proteins is return to self.batch_generate()
# '''

In [ ]:
# '''
#     We are inside iterative_sampling_raw
# '''
# proteins = [esm_protein]
# configs = [sequence_prediction_config]
# input_tokens = [model.encode(protein) for protein in proteins]
# ## If you have coordinates -> structure tokens are also computed

# output_tokens_list = model.batch_generate(input_tokens, configs)

# '''
#     iterative decoding happens inside batch_generate.
#     Initially its sequnce - real decoing happens when its ESM protein Tensor
#     then the iterative sample tokens is the real deal
# '''

# raw_proteins: list[ESMProtein | ESMProteinError] = []
# for output_tokens in output_tokens_list:
#     if isinstance(output_tokens, ESMProteinTensor):
#         raw_proteins.append(model.decode(output_tokens))
#     elif isinstance(output_tokens, ESMProteinError):
#         raw_proteins.append(output_tokens)
#     else:
#         raise ValueError(f"Unknown output type {type(output_tokens)}")

# for input_protein, raw_protein, config in zip(proteins, raw_proteins, configs):
#     if isinstance(raw_protein, ESMProteinError):
#         # If this generation errored out.
#         continue
#     if config.track not in ["function", "residue_annotations"]:
#         # Function and residue annotation encoding/decoding is lossy
#         # There is no guarantee that decoding encoded tokens will yield the same input
#         raw_protein.function_annotations = input_protein.function_annotations

# '''
#     in raw_proteins we get ESMProteins -> sequences 
# '''

In [ ]:
# '''
#  batch generate with tokens uses iterative_sampling_tokens
# '''
# proteins = [esm_protein]
# configs = [sequence_prediction_config]
# input_tokens = [model.encode(protein) for protein in proteins]

# inputs = input_tokens

# assert len(inputs) == len(
#             configs
#         ), "Must have the same number of prompts and configs."

# if inputs == []:
#     raise Exception('Empty list')

# # Make sure prompts are of the same type.
# t = type(inputs[0])
# for i in range(1, len(inputs)):
#     assert isinstance(inputs[i], t), (
#         "Prompts must have the same type. Got "
#         f"{t.__name__ and type(inputs[i]).__name__} instead."
#     )

# if isinstance(inputs[0], ESMProtein):
#     print('We go with iterative_sampling_raw')
#     proteins =  iterative_sampling_raw(model, inputs, configs)  # type: ignore
# elif isinstance(inputs[0], ESMProteinTensor):
#     print('We go with iterative_sampling_tokens')
#     proteins =  iterative_sampling_tokens(
#         model,
#         inputs,  # type: ignore
#         configs,
#         model.tokenizers,  # type: ignore
#     )
# else:
#     raise ValueError("Input must be an ESMProtein or ESMProteinTensor")

In [ ]:
# '''
#  just decode using model.decode
# '''
# model.decode(proteins[0])

In [ ]:
# from esm.utils.sampling import (
#     _BatchedESMProteinTensor,
#     get_sampling_mask,
#     sample_function_logits,
#     sample_logits,
#     sample_residue_annotation_logits,
#     sample_sasa_logits,
# )


In [ ]:
# def _batch_forward(client, protein):
#     # Forward pass
#     return client.logits(
#         protein,
#         LogitsConfig(
#             sequence=True,
#             structure=True,
#             secondary_structure=True,
#             sasa=True,
#             function=True,
#             residue_annotations=True,
#             return_embeddings=True,
#         ),
#     )

# '''
#     inside iterative_sample_tokens -> where magic happens
#     return ESMProteinTensor
# '''
# import attr

# proteins = [esm_protein]
# configs = [sequence_prediction_config]
# input_tokens = [model.encode(protein) for protein in proteins]
# tokenizers =  model.tokenizers

# devices = set([t.device for t in input_tokens])
# if len(devices) > 1:
#     raise AttributeError(f"Input tokens on multiple devices {devices}")

# sampled_tokens = [attr.evolve(tokens) for tokens in input_tokens]
# ## just ends up making a copy

# # Clear structure tokens if user would like to condition only on coordinates.
# for tokens, config in zip(sampled_tokens, configs):
#     if config.condition_on_coordinates_only and tokens.coordinates is not None:
#         print('We enter clear structure tokens')
#         tokens.structure = None

# # Total sequence lengths.
# sequence_lengths = [len(tokens) for tokens in sampled_tokens]
# # Figure out the number of tokens to be sampled for each prompt.
# total_to_sample = []
# for protein, config in zip(sampled_tokens, configs):
#     track = config.track

#     if getattr(protein, track) is None:
#         # We need to sample the entire track.
#         num_sampling_steps = _get_non_special_tokens(protein, tokenizers)
#         print(f'num_sampling_steps {num_sampling_steps}')
#         raise Exception('we wont eneter here')
#     else:
#         masked = _get_masked_positions(
#             track, getattr(protein, track), getattr(tokenizers, track).mask_token_id
#         )
#         num_sampling_steps = torch.sum(masked).item()

#     total_to_sample.append(num_sampling_steps)

#     # Users might over-specify the number of sampling steps for a given prompt
#     # TODO: Give a warning about mismatched num_steps and number of masks.

#     if (num_sampling_steps > 0) and (num_sampling_steps < config.num_steps):
#         config.num_steps = int(num_sampling_steps)

# max_num_steps = max([config.num_steps for config in configs])

# print(total_to_sample, max_num_steps)

# batched_tokens = _stack_protein_tensors(
#     sampled_tokens, sequence_lengths, tokenizers, devices.pop()
# )

# # Remember sampled prompts that has somehow errored out.
# errors: dict[int, ESMProteinError] = {}

# # Decode
# disable_tqdm = bool(os.environ.get("DISABLE_ITERATIVE_SAMPLING_TQDM", False))
# for t in tqdm(range(max_num_steps), disable=disable_tqdm):
#     forward_out = _batch_forward(model, batched_tokens)

#     for i, config in enumerate(configs):  # B

#         if i in errors:
#             # This prompts has errored out in previous steps.
#             # Skip.
#             continue

#         if config.track in ["coordinates", "residue_annotations"]:
#             errors[i] = ESMProteinError(
#                 error_code=500,
#                 error_msg=f"Iterative sampling {config.track} is not supported.",
#             )
#             continue

#         if t >= config.num_steps:
#             # Done sampling for this row.
#             continue

#         per_prompt_cur_sampled = _BatchedESMProteinTensor.from_protein_tensor(
#             batched_tokens.slice(i)
#         )

#         per_prompt_forward_out = _slice_tensor_dataclass(
#                 forward_out, i, keep_dim=True
#             )
        
#         per_prompt_forward_out = _trim_sequence_tensor_dataclass(
#             per_prompt_forward_out,
#             # Note(jungong) : we can not smiply use sequence_lenths[i] here,
#             # what we want is for the sequence length of the logits to match
#             # that of the prompt, which may or may not be padded, depending on
#             # whether the padding was done locally with the open source model
#             # (where per_prompt_cur_sampled is already padded) or by
#             # BatchedESM3ModelRunner (where per_prompt_cur_sampled is not padded).
#             len(per_prompt_cur_sampled),
#         )

#         if config.temperature_annealing:
#             temperature = _get_annealed_temperature(
#                 t, config.num_steps, config.temperature
#             )
#         else:
#             temperature = config.temperature

#         track_sample_config = SamplingTrackConfig()
#         track_sample_config.invalid_ids = config.invalid_ids
#         track_sample_config.temperature = temperature
#         track_sample_config.top_p = config.top_p
#         sampling_config = SamplingConfig(**{config.track: track_sample_config})  # type: ignore

#         # Sampling has to be done per-prompt, since sampling configs
#         # are likely be different for different prompts.
#         per_prompt_forward_and_sample_output = _sample_per_prompt(
#             per_prompt_cur_sampled,
#             per_prompt_forward_out,
#             sampling_config,
#             tokenizers,
#             decode_sasa_tokens=False,
#         )

#         # All positions sampled after _sample_per_prompt() above.
#         # (B, L) & (B, L, D)
#         per_prompt_new_sampled = per_prompt_forward_and_sample_output.protein_tensor

#         # Find the positions we should sample this round.
#         assert per_prompt_forward_and_sample_output.entropy is not None
#         try:
#             where_to_sample = _get_iterative_sampling_mask_for_prompt_and_step(
#                 per_prompt_cur_sampled,
#                 torch.tensor(sequence_lengths[i]),
#                 torch.tensor(total_to_sample[i]),
#                 t,
#                 per_prompt_forward_and_sample_output.entropy,
#                 config,
#                 tokenizers,
#             )
#         except ValueError as e:
#             errors[i] = ESMProteinError(error_code=500, error_msg=str(e))
#             continue

#         where_to_sample.to(input_tokens[0].device)

#         old_track_samples = getattr(per_prompt_cur_sampled, config.track)
#         new_track_samples = getattr(per_prompt_new_sampled, config.track)

#         # Iterative sampling by picking the tokens sampled this round
#         # from new_track_samples to old_track_samples.
#         new_track_samples = torch.where(
#             where_to_sample, new_track_samples, old_track_samples
#         )

#         # Update the corresponding row with new data.
#         getattr(batched_tokens, config.track)[i, ...] = new_track_samples[0]

# # Un-pack to a list of single ProteinTypes.
# output_tokens = [
#     batched_tokens.slice(i, sequence_len=sequence_lengths[i])
#     if i not in errors
#     else errors[i]
#     for i in range(len(input_tokens))
# ]

# # Do not update tracks that were not sampled (e.g. keep None instead of masks)
# for inputs, outputs, config in zip(input_tokens, output_tokens, configs):
#     if isinstance(outputs, ESMProteinError):
#         continue

#     # First restore coordinates field.
#     # We know coordinates can never be iteratively sampled.
#     setattr(outputs, "coordinates", getattr(inputs, "coordinates"))
#     # Maybe restore all the other fields.
#     for f in attr.fields(SamplingConfig):
#         if "embedding" in f.name or f.name == "return_hidden_states":
#             continue
#         if f.name != config.track:
#             setattr(outputs, f.name, getattr(inputs, f.name))

#     break

In [ ]:
# model.decode(output_tokens[0])

In [ ]:
# ''.join(np.array(list(model.decode(output_tokens[0]).sequence))[T_gxps_atc])

In [ ]:
# class DPOTrainer:
#     def __init__(self, model, model_ref, device='cpu') -> None:
#         self.device = device

#         if self.device == 'gpu':
#             self.model = model.to("cuda")
#             self.model_ref = model_ref.to("cuda")
#         else:
#             self.model = model.to("cpu")
#             self.model_ref = model_ref.to("cpu")

#         # self.model_ref.eval()
#         # for pm in self.model_ref.parameters():
#         #     pm.requires_grad = False

#         for name, param in self.model.named_parameters():
#             if name in [
#                 "encoder.sequence_embed.weight", 
#                 "output_heads.sequence_head.0.weight", 
#                 "output_heads.sequence_head.0.bias",
#                 "output_heads.sequence_head.2.weight",
#                 "output_heads.sequence_head.2.bias",
#                 "output_heads.sequence_head.3.weight",
#                 "output_heads.sequence_head.3.bias"
#             ]:
#                 param.requires_grad = True
#             else:
#                 param.requires_grad = False

#         ## need to add peft here ? Do we ?

#     def generate_batch(self, sequence_prompt, structure_prompt, batch_size):
#         sequence_prediction_config = GenerationConfig(
#             track="sequence", 
#             num_steps=sequence_prompt.count("_") // 2, 
#             temperature=0.5
#         )
#         esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=structure_prompt)
        
#         generated_proteins = []
#         for _ in tqdm(range(batch_size)):
#             generated_protein = self.model.generate(esm_protein, sequence_prediction_config)
#             generated_proteins.append(generated_protein.sequence)

#         return generated_proteins
    
#     def get_mask_positions(self, sequence_prompt, mask_var='_'):
#         positions = []
#         for i in range(len(sequence_prompt)):
#             if sequence_prompt[i] == mask_var:
#                 positions.append(i)

#         return positions
    
#     def __get_n_mutations(self, sequence):
#         base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
#         assert len(base) == len(sequence)

#         count = 0
#         for aa1, aa2 in zip(base, sequence):
#             if aa1 != aa2:
#                 count += 1

#         return count
    
#     def get_property_batch(self, batch):
#         for item in batch:
#             item['property'] = self.__get_n_mutations(item['masked_sequence'])

#         return batch
    
#     def get_log_prob(self, sequence):
#         esm_protein = ESMProtein(sequence=sequence)
#         esm_tensor = self.model.encode(esm_protein)

#         results = self.model.logits(
#             esm_tensor, LogitsConfig(sequence=True, return_embeddings=False)
#         )

#         logits = results.logits.sequence
#         log_prob = torch.log_softmax(logits[0, 1:-1, :33], dim=-1)

#         return log_prob.cpu().numpy()
    
#     def get_log_likelihood(self, batch, masked_positions, mask_var='_'):
#         '''
#             for now mask each pos in mask
#         '''
#         for item in batch:
#             sum_log = 0
#             for pos in masked_positions:
#                 masked_query = list(item['sequence'])
#                 assert mask_var not in masked_query
#                 masked_query[pos] = mask_var
#                 masked_query = ''.join(masked_query)
#                 log_prob = self.get_log_prob(sequence=masked_query)
#                 assert log_prob.shape[0] == len(item['sequence'])

#                 prob_pos = log_prob[pos, self.model.tokenizers.sequence.convert_tokens_to_ids(item['sequence'][pos])]
#                 sum_log += prob_pos                

#             item['log_likelihood'] = sum_log/len(masked_positions)

#         return batch
    
#     def create_pairs(self, batch):
        
#         pair_dataset = []
#         for item_1 in batch[:-1]:
#             for item_2 in batch[1:]:
#                 if item_1['property'] < item_2['property'] and item_1['log_likelihood'] < item_2['log_likelihood']:
#                     pair_dataset.append((item_1, item_2))
#                 elif item_1['property'] > item_2['property'] and item_1['log_likelihood'] > item_2['log_likelihood']:
#                     pair_dataset.append((item_2, item_1))

#         _, ax = plt.subplots(1, 2, figsize=(7, 3), layout='constrained')
#         ax[0].hist([[item[0]['property'] for item in pair_dataset], 
#                     [item[1]['property'] for item in pair_dataset]], label=['pos', 'neg'])
#         ax[1].hist([[item[0]['log_likelihood'] for item in pair_dataset],
#                     [item[1]['log_likelihood'] for item in pair_dataset]], label=['pos', 'neg'])
#         ax[0].legend()
#         ax[1].legend()
#         plt.show()

#         return pair_dataset
    
#     def get_policy(self, batch):

#         proteins = [ESMProtein(sequence=item['generated_sequence'], 
#                                    coordinates=item['structure_prompt']) 
#                                    for item in batch]
        
#         input_tokens = [self.model.encode(protein) for protein in proteins]

#         print(input_tokens)
        
    
#     def dpo_paired_loss(self, batch):

#         self.get_policy(batch)
    



#     def train(self, sequence_prompt, structure_prompt):

#         self.model.train()

#         mask_positions = self.get_mask_positions(sequence_prompt=sequence_prompt)

#         epochs = 3
#         batch_size = 8
#         batch_dataset = []
#         for epoch in range(epochs):
#             batch_seq = self.generate_batch(sequence_prompt=sequence_prompt, 
#                                         structure_prompt=structure_prompt,
#                                         batch_size=batch_size)
            
#             for _seq in batch_seq:
#                 batch_dataset.append({
#                     'generated_sequence': _seq,
#                     'masked_sequence': ''.join([s for _i, s in enumerate(_seq) if _i in mask_positions]),
#                     'masked_positions': mask_positions, ## These are the same
#                     'sequence_prompt': sequence_prompt, ## These are the same
#                     'structure_prompt': structure_prompt ## These are the same        
#                 })

#             batch_dataset = self.get_property_batch(batch_dataset)

#             # batch_dataset = self.get_log_likelihood(batch_dataset, mask_positions)
#             # pair_dataset = self.create_pairs(batch_dataset)

#             return batch_dataset

#             self.dpo_paired_loss(batch_dataset)
                
#             break

In [ ]:
# class DPOTrainer:
#     def __init__(self, model, model_ref, device='cpu') -> None:
#         self.device = device

#         # if self.device == 'gpu':
#         #     self.model = model.to("cuda:0")
#         #     self.model_ref = model_ref.to("cuda:1")
#         # else:
#         #     self.model = model.to("cpu")
#         #     self.model_ref = model_ref.to("cpu")

#         self.model = model
#         self.model_ref =  model_ref

#         self.model_ref.eval()
#         for pm in self.model_ref.parameters():
#             pm.requires_grad = False

#         for name, param in self.model.named_parameters():
#             if name in [
#                 "encoder.sequence_embed.weight", 
#                 "output_heads.sequence_head.0.weight", 
#                 "output_heads.sequence_head.0.bias",
#                 "output_heads.sequence_head.2.weight",
#                 "output_heads.sequence_head.2.bias",
#                 "output_heads.sequence_head.3.weight",
#                 "output_heads.sequence_head.3.bias"
#             ]:
#                 param.requires_grad = True
#             else:
#                 param.requires_grad = False

#         ## need to add peft here ? Do we ?
    
#     def print_trainable_parameters(self, model):
#         trainable_params = 0
#         all_param = 0
#         for _, param in model.named_parameters():
#             all_param += param.numel()
#             if param.requires_grad:
#                 trainable_params += param.numel()
#         print(
#             f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
#         )

#     def generate_batch(self, sequence_prompt, structure_prompt, batch_size, ref=False):
#         sequence_prediction_config = GenerationConfig(
#             track="sequence", 
#             num_steps=sequence_prompt.count("_") // 2, 
#             temperature=0.5
#         )
#         esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=structure_prompt)
        
#         generated_proteins = []
#         for _ in tqdm(range(batch_size)):
#             generated_protein = self.model.generate(esm_protein, sequence_prediction_config)
#             generated_proteins.append(generated_protein.sequence)

#         return generated_proteins
    
#     def get_mask_positions(self, sequence_prompt, mask_var='_'):
#         positions = []
#         for i in range(len(sequence_prompt)):
#             if sequence_prompt[i] == mask_var:
#                 positions.append(i)

#         return np.array(positions)
    
#     def __get_n_mutations(self, sequence):
#         base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
#         assert len(base) == len(sequence)

#         count = 0
#         for aa1, aa2 in zip(base, sequence):
#             if aa1 != aa2:
#                 count += 1

#         return -1*count
    
#     def get_property_batch(self, batch):
#         for item in batch:
#             item['property'] = self.__get_n_mutations(item['masked_sequence'])

#         return batch
    
#     def get_PLL(self, item, ref=False):
#         '''
#             We always feed in the sequence prompt
#             We get Log Prob(A_C) -  entire T region is masked
#         '''
#         proteins = [ESMProtein(sequence=item['sequence_prompt'], 
#                         coordinates=item['structure_prompt'])]
        
#         if ref:
#             input_tokens = [self.model_ref.encode(protein) for protein in proteins]
#             generated_tokens = self.model_ref.encode(ESMProtein(sequence=item['generated_sequence'], 
#                                                         coordinates=item['structure_prompt']))
#         else:
#             input_tokens = [self.model.encode(protein) for protein in proteins]
#             generated_tokens = self.model.encode(ESMProtein(sequence=item['generated_sequence'], 
#                                                         coordinates=item['structure_prompt']))
        
        
#         devices = set([t.device for t in input_tokens])
#         if len(devices) > 1:
#                 raise AttributeError(f"Input tokens on multiple devices {devices}")
#         sequence_lengths = [len(tokens) for tokens in input_tokens]
#         assert len(set(sequence_lengths)) == 1
        
#         if ref:
#             batched_tokens = _stack_protein_tensors(
#                     input_tokens, sequence_lengths, self.model_ref.tokenizers, self.model_ref.device
#                 )
#         else:
#             batched_tokens = _stack_protein_tensors(
#                     input_tokens, sequence_lengths, self.model.tokenizers, self.model.device
#                 )
        
#         if batched_tokens.coordinates is None:
#             per_res_plddt = None
#         else:
#             # 1.0 if all coordinates at specific indices have valid non-nan values.
#             per_res_plddt = batched_tokens.coordinates.isfinite().all(dim=-1).any(dim=-1).float()

#         if ref:
#             assert model_ref.device == batched_tokens.device
#             with (torch.no_grad(),
#                 torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
#                 output = self.model_ref.forward(
#                         sequence_tokens=batched_tokens.sequence,
#                         structure_tokens=batched_tokens.structure,
#                         ss8_tokens=batched_tokens.secondary_structure,
#                         sasa_tokens=batched_tokens.sasa,
#                         function_tokens=batched_tokens.function,
#                         residue_annotation_tokens=batched_tokens.residue_annotations,
#                         average_plddt=torch.tensor(1.0, device=batched_tokens.device),
#                         per_res_plddt=per_res_plddt,
#                         structure_coords=batched_tokens.coordinates,
#                         chain_id=None,
#                         sequence_id=None,
#                 )
#         else:
#             assert model.device == batched_tokens.device
#             with (torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
#                 output = self.model.forward(
#                         sequence_tokens=batched_tokens.sequence,
#                         structure_tokens=batched_tokens.structure,
#                         ss8_tokens=batched_tokens.secondary_structure,
#                         sasa_tokens=batched_tokens.sasa,
#                         function_tokens=batched_tokens.function,
#                         residue_annotation_tokens=batched_tokens.residue_annotations,
#                         average_plddt=torch.tensor(1.0, device=batched_tokens.device),
#                         per_res_plddt=per_res_plddt,
#                         structure_coords=batched_tokens.coordinates,
#                         chain_id=None,
#                         sequence_id=None,
#                 )
        
#         log_prob = torch.log_softmax(output.sequence_logits, dim=-1)

#         masked_log_prob = log_prob[:, item['masked_positions'] + 1, generated_tokens.sequence[item['masked_positions'] + 1]]
#         pll = torch.sum(masked_log_prob)

#         # print(item['masked_sequence'])
#         print(generated_tokens.sequence[item['masked_positions'] + 1])
#         print(masked_log_prob)
#         # print(pll)

#         return pll
         
#     def dpo_paired_loss(self, batch):

#         pll_policy = []
#         pll_ref = []
#         for item in batch:
#             pll_policy.append(self.get_PLL(item))
#             pll_ref.append(self.get_PLL(item, ref=True))
#             print(item['property'])

#         print([x['property'] for x in batch])
#         print(pll_policy)
#         print(pll_ref)

#         pll_ref = [x.to(self.model.device) for x in pll_ref]

#         print(pll_ref)

#         batch_size = len(batch)
#         assert batch_size > 1
#         beta = 0.01

#         loss = []

#         pairs_debug = []
#         for i in range(batch_size-1):
#             for j in range(1, batch_size):
#                 if batch[i]['property'] > batch[j]['property'] + 0:
#                     loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i] - pll_ref[i]) - (pll_policy[j] - pll_ref[j])) )
#                     # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i]) - (pll_policy[j])) )
#                     loss.append(loss_item)

#                     pairs_debug.append((batch[i]['property'], batch[j]['property'],
#                                         (pll_policy[i] - pll_ref[i]).item(), (pll_policy[j] - pll_ref[j]).item(),
#                                         loss_item.item()))
                    
#                 elif batch[j]['property'] > batch[i]['property'] + 0:
#                     loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j] - pll_ref[j]) - (pll_policy[i] - pll_ref[i])) )
#                     # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j]) - (pll_policy[i])) )
#                     loss.append(loss_item)

#                     pairs_debug.append((batch[j]['property'], batch[i]['property'],
#                                         (pll_policy[j] - pll_ref[j]).item(), (pll_policy[i] - pll_ref[i]).item(),
#                                         loss_item.item()))
                    
#         for _pair in pairs_debug:
#             print(f'pair :{_pair}')

#         if len(loss) > 0:
#             loss = torch.mean(torch.stack(loss))
#         else:
#             loss = torch.tensor(0.).to(self.model.device)
                    
#         return loss
 
#     def train(self, sequence_prompt, structure_prompt):

#         self.model.train()

#         self.print_trainable_parameters(self.model)
#         self.print_trainable_parameters(self.model_ref)

#         optimizer = torch.optim.AdamW(
#             self.model.parameters(),
#             lr=1e-3,
#             betas=(0.9, 0.98),
#             eps=1e-8,
#             weight_decay=0.1,
#         )

#         mask_positions = self.get_mask_positions(sequence_prompt=sequence_prompt)

#         epochs = 100
#         batch_size = 1
#         for epoch in range(epochs):
#             optimizer.zero_grad()

#             batch_dataset = []
#             print(f'epoch: {epoch}: Generating batch')
#             batch_seq = self.generate_batch(sequence_prompt=sequence_prompt, 
#                                         structure_prompt=structure_prompt,
#                                         batch_size=batch_size)
            
#             for _seq in batch_seq:
#                 batch_dataset.append({
#                     'generated_sequence': _seq,
#                     'masked_sequence': ''.join([s for _i, s in enumerate(_seq) if _i in mask_positions]),
#                     'masked_positions': mask_positions, ## These are the same
#                     'sequence_prompt': sequence_prompt, ## These are the same
#                     'structure_prompt': structure_prompt ## These are the same        
#                 })

#             A, C = sequence_prompt.split('_'*115)
#             base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
#             batch_dataset.append({
#                 'generated_sequence': A+base+C,
#                 'masked_sequence': base,
#                 'masked_positions': mask_positions, ## These are the same
#                 'sequence_prompt': sequence_prompt, ## These are the same
#                 'structure_prompt': structure_prompt ## These are the same  
#             })

#             batch_dataset = self.get_property_batch(batch_dataset)
            
#             print(f'epoch {epoch}: calculating loss')
#             loss = self.dpo_paired_loss(batch_dataset)
#             print(f'loss {loss.item()}')

#             if loss != 0:
#                 print(f'epoch {epoch}: optimizer step')
#                 loss.backward()
#                 optimizer.step()

#             torch.cuda.empty_cache()

### Online DPO

In [7]:
model = ESM3.from_pretrained("esm3_sm_open_v1").to('cuda')
# model_ref = ESM3.from_pretrained("esm3_sm_open_v1")

In [10]:
torch.cuda.memory_allocated()/1024**2

2692.4638671875

In [11]:
torch.cuda.memory_reserved()/1024**2

11156.0

In [12]:
class DPOTrainer:
    def __init__(self, model, model_ref, device='cpu') -> None:
        self.device = device

        # if self.device == 'gpu':
        #     self.model = model.to("cuda:0")
        #     self.model_ref = model_ref.to("cuda:1")
        # else:
        #     self.model = model.to("cpu")
        #     self.model_ref = model_ref.to("cpu")

        self.model = model
        self.model_ref =  model_ref

        self.model_ref.eval()
        for pm in self.model_ref.parameters():
            pm.requires_grad = False

        for name, param in self.model.named_parameters():
            if name in [
                "encoder.sequence_embed.weight", 
                "output_heads.sequence_head.0.weight", 
                "output_heads.sequence_head.0.bias",
                "output_heads.sequence_head.2.weight",
                "output_heads.sequence_head.2.bias",
                "output_heads.sequence_head.3.weight",
                "output_heads.sequence_head.3.bias"
            ]:
                param.requires_grad = True
            else:
                param.requires_grad = False

        ## need to add peft here ? Do we ?
    
    def print_trainable_parameters(self, model):
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
        )

    def generate_sequences(self, sequence_prompt, structure_prompt, size, ref=False):
        sequence_prediction_config = GenerationConfig(
            track="sequence", 
            num_steps=sequence_prompt.count("_") // 2, 
            temperature=0.5
        )
        esm_protein = ESMProtein(sequence=sequence_prompt, coordinates=structure_prompt)
        
        generated_proteins = []
        for _ in tqdm(range(size)):
            generated_protein = self.model.generate(esm_protein, sequence_prediction_config)
            generated_proteins.append(generated_protein.sequence)

        return generated_proteins
    
    def create_dataset(self, sequence_prompt, structure_prompt, mask_positions, size):

        generated_sequences = self.generate_sequences(sequence_prompt, structure_prompt, size)

        dataset = []
        for _seq in generated_sequences:
            dataset.append({
                'generated_sequence': _seq,
                'masked_sequence': ''.join([s for _i, s in enumerate(_seq) if _i in mask_positions]),
                'masked_positions': mask_positions, ## These are the same
                'sequence_prompt': sequence_prompt, ## These are the same
                'structure_prompt': structure_prompt, ## These are the same   
            })

        for item in dataset:
            item['property'] = self.__get_n_mutations(item['masked_sequence'])
        
        return dataset
    
    def get_mask_positions(self, sequence_prompt, mask_var='_'):
        positions = []
        for i in range(len(sequence_prompt)):
            if sequence_prompt[i] == mask_var:
                positions.append(i)

        return np.array(positions)
    
    def __get_n_mutations(self, sequence):
        base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
        assert len(base) == len(sequence)

        count = 0
        for aa1, aa2 in zip(base, sequence):
            if aa1 != aa2:
                count += 1

        return -1*count
    
    # def get_property_batch(self, batch):
    #     for item in batch:
    #         item['property'] = self.__get_n_mutations(item['masked_sequence'])

    #     return batch
    
    def get_PLL(self, item, ref=False):
        '''
            We always feed in the sequence prompt
            We get Log Prob(A_C) -  entire T region is masked
        '''
        proteins = [ESMProtein(sequence=item['sequence_prompt'], 
                        coordinates=item['structure_prompt'])]
        
        if ref:
            input_tokens = [self.model_ref.encode(protein) for protein in proteins]
            generated_tokens = self.model_ref.encode(ESMProtein(sequence=item['generated_sequence'], 
                                                        coordinates=item['structure_prompt']))
        else:
            input_tokens = [self.model.encode(protein) for protein in proteins]
            generated_tokens = self.model.encode(ESMProtein(sequence=item['generated_sequence'], 
                                                        coordinates=item['structure_prompt']))
        
        
        devices = set([t.device for t in input_tokens])
        if len(devices) > 1:
                raise AttributeError(f"Input tokens on multiple devices {devices}")
        sequence_lengths = [len(tokens) for tokens in input_tokens]
        assert len(set(sequence_lengths)) == 1
        
        if ref:
            batched_tokens = _stack_protein_tensors(
                    input_tokens, sequence_lengths, self.model_ref.tokenizers, self.model_ref.device
                )
        else:
            batched_tokens = _stack_protein_tensors(
                    input_tokens, sequence_lengths, self.model.tokenizers, self.model.device
                )
        
        if batched_tokens.coordinates is None:
            per_res_plddt = None
        else:
            # 1.0 if all coordinates at specific indices have valid non-nan values.
            per_res_plddt = batched_tokens.coordinates.isfinite().all(dim=-1).any(dim=-1).float()

        if ref:
            assert model_ref.device == batched_tokens.device
            with (torch.no_grad(),
                torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
                output = self.model_ref.forward(
                        sequence_tokens=batched_tokens.sequence,
                        structure_tokens=batched_tokens.structure,
                        ss8_tokens=batched_tokens.secondary_structure,
                        sasa_tokens=batched_tokens.sasa,
                        function_tokens=batched_tokens.function,
                        residue_annotation_tokens=batched_tokens.residue_annotations,
                        average_plddt=torch.tensor(1.0, device=batched_tokens.device),
                        per_res_plddt=per_res_plddt,
                        structure_coords=batched_tokens.coordinates,
                        chain_id=None,
                        sequence_id=None,
                )
        else:
            assert model.device == batched_tokens.device
            with (torch.autocast(enabled=True, device_type=torch.device(batched_tokens.device).type, dtype=torch.bfloat16)):
                output = self.model.forward(
                        sequence_tokens=batched_tokens.sequence,
                        structure_tokens=batched_tokens.structure,
                        ss8_tokens=batched_tokens.secondary_structure,
                        sasa_tokens=batched_tokens.sasa,
                        function_tokens=batched_tokens.function,
                        residue_annotation_tokens=batched_tokens.residue_annotations,
                        average_plddt=torch.tensor(1.0, device=batched_tokens.device),
                        per_res_plddt=per_res_plddt,
                        structure_coords=batched_tokens.coordinates,
                        chain_id=None,
                        sequence_id=None,
                )
        
        log_prob = torch.log_softmax(output.sequence_logits, dim=-1)

        masked_log_prob = log_prob[:, item['masked_positions'] + 1, generated_tokens.sequence[item['masked_positions'] + 1]]
        pll = torch.sum(masked_log_prob)

        # print(item['masked_sequence'])
        print(generated_tokens.sequence[item['masked_positions'] + 1])
        print(masked_log_prob)
        # print(pll)

        return pll
         
    def dpo_paired_loss(self, batch):

        pll_policy = []
        pll_ref = []
        for item in batch:
            pll_policy.append(self.get_PLL(item))
            pll_ref.append(self.get_PLL(item, ref=True))
            print(item['property'])

        print([x['property'] for x in batch])
        print(pll_policy)
        print(pll_ref)

        pll_ref = [x.to(self.model.device) for x in pll_ref]

        print(pll_ref)

        batch_size = len(batch)
        assert batch_size > 1
        beta = 0.01

        loss = []

        pairs_debug = []
        for i in range(batch_size-1):
            for j in range(1, batch_size):
                if batch[i]['property'] > batch[j]['property'] + 0:
                    loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i] - pll_ref[i]) - (pll_policy[j] - pll_ref[j])) )
                    # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[i]) - (pll_policy[j])) )
                    loss.append(loss_item)

                    pairs_debug.append((batch[i]['property'], batch[j]['property'],
                                        (pll_policy[i] - pll_ref[i]).item(), (pll_policy[j] - pll_ref[j]).item(),
                                        loss_item.item()))
                    
                elif batch[j]['property'] > batch[i]['property'] + 0:
                    loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j] - pll_ref[j]) - (pll_policy[i] - pll_ref[i])) )
                    # loss_item = -torch.nn.functional.logsigmoid( beta * ((pll_policy[j]) - (pll_policy[i])) )
                    loss.append(loss_item)

                    pairs_debug.append((batch[j]['property'], batch[i]['property'],
                                        (pll_policy[j] - pll_ref[j]).item(), (pll_policy[i] - pll_ref[i]).item(),
                                        loss_item.item()))
                    
        for _pair in pairs_debug:
            print(f'pair :{_pair}')

        if len(loss) > 0:
            loss = torch.mean(torch.stack(loss))
        else:
            loss = torch.tensor(0.).to(self.model.device)
                    
        return loss
 
    def train(self, sequence_prompt, structure_prompt):

        self.model.train()

        self.print_trainable_parameters(self.model)
        self.print_trainable_parameters(self.model_ref)

        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=1e-3,
            betas=(0.9, 0.98),
            eps=1e-8,
            weight_decay=0.1,
        )

        mask_positions = self.get_mask_positions(sequence_prompt=sequence_prompt)

        dataset = self.create_dataset(sequence_prompt, structure_prompt,
                                      mask_positions, 10)
        
        return dataset

        # epochs = 100
        # batch_size = 1
        # for epoch in range(epochs):
        #     optimizer.zero_grad()

        #     # A, C = sequence_prompt.split('_'*115)
        #     # base = 'APSEDAYPRATYEAPEGETEQLLAGIWMDLLQVDRVGRHDSFFELGGHSLLAVRLLGRLRQHGLGLQMRDLFEAPVLAELATRLRPYQPLEVPANGITPDTTVLTPEMLPLVTLS'
        #     # batch_dataset.append({
        #     #     'generated_sequence': A+base+C,
        #     #     'masked_sequence': base,
        #     #     'masked_positions': mask_positions, ## These are the same
        #     #     'sequence_prompt': sequence_prompt, ## These are the same
        #     #     'structure_prompt': structure_prompt ## These are the same  
        #     # })

        #     batch_dataset = self.get_property_batch(batch_dataset)
            
        #     print(f'epoch {epoch}: calculating loss')
        #     loss = self.dpo_paired_loss(batch_dataset)
        #     print(f'loss {loss.item()}')

        #     if loss != 0:
        #         print(f'epoch {epoch}: optimizer step')
        #         loss.backward()
        #         optimizer.step()

        #     torch.cuda.empty_cache()

In [13]:
protein = ProteinChain.from_pdb('../../Data/gxps/gxps_ATC_AF.pdb')

sequence_prompt = ''.join([protein[i].sequence if i in A_gxps_atc + C_gxps_atc else '_' for i in range(len(protein))])
# structure_prompt = torch.full((len(sequence_prompt), 37, 3), np.nan)
# structure_prompt[T_gxps_atc] = torch.tensor(protein.atom37_positions)[T_gxps_atc]
# structure_prompt = torch.tensor(protein.atom37_positions)[T_gxps_atc[0]-50:T_gxps_atc[-1]+50]

# sequence_prompt = ''.join([s for i, s in enumerate(sequence_prompt) if i in list(range(T_gxps_atc[0]-50,T_gxps_atc[-1]+50))])

In [14]:
len(sequence_prompt), sequence_prompt

(1034,
 'VCVHQLFEQQIEKTPDAIAVIYENQTLSYAELNARANRLAHQLIALGVAPDQRVAICVTRSLARIIGLLAVLKAGGAYVPLDPAYPGERLAYMLTDATPVILMADNVGRAALSEDILATLTVLDPNTLLEQPDHNPQVSGLTPQHLAYVIYTSGSTGRPKGVMIEHRSVVNLTLTQITQFDVCATSRMLQFASFGFDASVWEIMMALSCGAMLVIPTETVRQDPQRLWRYLEEQAITHACLTPAMFHDGTDLPAIAIKPTLIFAGEAPSPALFQALCSRADLFNAYGPTEITVCATTWDCPADYTGGVIPIGSPVANKRLYLLDEHRQPVPLGTVGELYIGGVGVARGYLNRPELTAERFLNDPFSDETNARMYRAGDLARYLPDGNLVFVGRNDQQVKIRGFRIEPGEIEARLVEHSEVSEALVLALGDGQDKRLVAYVVALADDGLATKLREHLSDILPDYMIPAAFVRLDAFPLTPNGKLDRRSLP___________________________________________________________________________________________________________________QAEIDRIVEQVPGGIANIQDIYALSPLQDGILFHHLLANEGDPYLLITQQAFADRPLLNRYLAAVQQVVDRHDILRTAFIWEGLSVPAQVICRQAPLSVTELTLNPADGAISNQLAQRFDPRRHRIDLNQAPLLRFVVAQESDGRWILLQLLHHLIGDHTTLEVMNSEVQACLLGQMDSLPAPVPFRHLVAQARQGVSQAEHTRFFTDMLAEVDEPTLLFGLAEAHHDGSQVTESHRMLTAGLNERLRGQARRLGVSVAALCHLAWAQVLSRTSGQTQVVFGTVLFGRMQAGEGSDSGMGLFINTLPLRLDIDNTPVRDSVRAAHSRLAGLLEHEHASLALAQRCSGVESGTPLFNALLNYRHNTQPVTPDEIVSGIEFLGAQERTN

In [15]:
len(sequence_prompt) == structure_prompt.shape[0]

NameError: name 'structure_prompt' is not defined

In [16]:
model_ref = model
dpo_trainer = DPOTrainer(model, model_ref, device='gpu')

In [17]:
os.environ["DISABLE_ITERATIVE_SAMPLING_TQDM"] = "True"

In [18]:
dataset = dpo_trainer.train(sequence_prompt=sequence_prompt, structure_prompt=None)

trainable params: 2560576 || all params: 1401735748 || trainable%: 0.18
trainable params: 2560576 || all params: 1401735748 || trainable%: 0.18


100%|██████████████████████████████████████████████████████| 10/10 [00:59<00:00,  5.97s/it]


In [ ]:
dataset

In [ ]:
def create_pairs(dataset):
    
    pair_dataset = []
    for item_1 in batch[:-1]:
        for item_2 in batch[1:]:
            if item_1['property'] < item_2['property'] and item_1['log_likelihood'] < item_2['log_likelihood']:
                pair_dataset.append((item_1, item_2))
            elif item_1['property'] > item_2['property'] and item_1['log_likelihood'] > item_2['log_likelihood']:
                pair_dataset.append((item_2, item_1))

    # _, ax = plt.subplots(1, 2, figsize=(7, 3), layout='constrained')
    # ax[0].hist([[item[0]['property'] for item in pair_dataset], 
    #             [item[1]['property'] for item in pair_dataset]], label=['pos', 'neg'])
    # ax[1].hist([[item[0]['log_likelihood'] for item in pair_dataset],
    #             [item[1]['log_likelihood'] for item in pair_dataset]], label=['pos', 'neg'])
    # ax[0].legend()
    # ax[1].legend()
    # plt.show()

    return pair_dataset